## Neural Network approach to simple least squares regression

In [327]:
import torch
import bokeh
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Label
import numpy as np
import tqdm
print(f"Using Torch version {torch.__version__}.  CUDA is {'available' if torch.cuda.is_available() else 'not available'}.")
print(f"Using bokeh version {bokeh.__version__}.")
output_notebook()

Using Torch version 1.13.1.post200.  CUDA is available.
Using bokeh version 3.1.0.


Loading BokehJS ...

We will use the simulated multivariate regression data for this demonstration

In [9]:
data = np.genfromtxt("data.csv",delimiter=",",skip_header=1)
Y = data[:,1].reshape(-1,1)
X = data[:,2:]


The OLS class definition has a single linear layer with no activation function.  The forward method is the same as the linear regression model.

In [228]:
class OLS(torch.nn.Module):
    """A simple 2x1 ordinary least squares model."""
    def __init__(self):
        super().__init__()
        self.linear=torch.nn.Linear(2,1,dtype=torch.float64)
    
    def forward(self,x):
        return self.linear(x)
    
    
model = OLS()
print(model)
for name,param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

OLS(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Layer: linear.weight | Size: torch.Size([1, 2]) | Values : tensor([[ 0.3777, -0.1856]], dtype=torch.float64, grad_fn=<SliceBackward0>) 

Layer: linear.bias | Size: torch.Size([1]) | Values : tensor([0.4363], dtype=torch.float64, grad_fn=<SliceBackward0>) 



In [261]:
Xt = torch.from_numpy(X)
Yt = torch.from_numpy(Y)
loss_fun = torch.nn.functional.mse_loss
optimizer = torch.optim.SGD(model.parameters(),lr=.01)


Here we just check that the loss function computed by the mse function is the same as the one computed by hand.

In [262]:
# loss computed by loss_func
loss = loss_fun(model(Xt),Yt)

# loss computed by hand
weights = list(model.parameters())
w = weights[0].detach()
b = weights[1].detach()
handloss = 1/Xt.shape[0]*(((Xt @ w.T + b) - Yt)**2).sum().item()

print(f"From function: {loss.item():.4f} | By hand: {handloss:.4f}")


From function: 1.0365 | By hand: 1.0365


The "training loop" is one step through the data in the stochastic gradient descent

In [232]:
def train(model, loss_fun, optimizer, Xt, Yt, debug=False):
    """One step through the training loop"""
    
    # forward pass
    predicted = model(Xt)
    
    # compute the loss
    loss = loss_fun(predicted, Yt)
    
    # reset the gradient calculations
    optimizer.zero_grad()

    # compute the gradients by backward propogation
    loss.backward()

    # Show that the gradients are what they should be
    if debug:
        print(f"Loss: {loss.item():.4f}")
        print(f"w gradients are: {model.linear.weight.grad[:2]}")
        print(f"b gradients are: {model.linear.bias.grad}")
        print(f"By hand: {2/Xt.shape[0]*((predicted - Yt).T @ Xt)}")
        print(f"By hand: {2/Xt.shape[0]*((predicted - Yt).sum())}")
        
    # adjust the weights
    optimizer.step()
    
    return loss.item()

    


This utility function returns a plot of loss over time.  You can set a learning rate and a threshold for ending the training loop.

In [328]:
def plot_loss(model, data, target, learning_rate=.01, threshold=1e-6, max_iter=100000):
    """Run the model and collect the losses; return a figure"""
    
    
    loss_fun = torch.nn.functional.mse_loss
    optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

    losses = []
    prior_loss=1000000
    for i in tqdm.tqdm(range(max_iter)):
        loss = train(model,loss_fun,optimizer,Xt,Yt,debug=False)
        losses.append(loss)
        if abs(loss-prior_loss) < threshold:
            break
        prior_loss = loss

    w = model.linear.weight.detach().numpy()
    b = model.linear.bias.detach().numpy()

    label = Label(text=f" Final Weights\n w0={w[0,0]:.2f}\n w1={w[0,1]:.2f}\n b={b[0]:.2f}",x=400,y=440,x_units='screen',y_units='screen')
    
    f=figure(title=f"Loss over time: lr={learning_rate} threshold={threshold}",x_axis_label="Epoch",y_axis_label="Loss")
    f.add_layout(label)
    f.line(x=list(range(len(losses))),y=losses)
    
    return f


In [330]:
model=OLS()
show(plot_loss(model, Xt, Yt,learning_rate=.01,threshold=1e-6))

  1%|▏         | 1352/100000 [00:00<00:17, 5544.04it/s]
